# Import Modules

In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [40]:
# Download NLTK resources (uncomment if not already downloaded)
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
# nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\otto\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\otto\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\otto\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\otto\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Load the data set

In [2]:
rating_df = pd.read_csv("../data/rating.csv")
domains_location_df = pd.read_csv("../data/domains_location.csv")
traffic_data_df = pd.read_csv("../data/traffic.csv")

In [3]:
rating_df.head()

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,article,title_sentiment
0,81664,NaN,Forbes,"Elizabeth Brownfield, Contributor, \n Elizabet...",superstar chef yannick alléno brings refined f...,Now open in Mayfair at Four Seasons Hotel Lond...,https://www.forbes.com/sites/elizabethbrownfie...,https://imageio.forbes.com/specials-images/ima...,2023-11-01 03:27:21.000000,"Pavyllon London, at Four Seasons Hotel London ...",Monaco,"pavyllon london, at four seasons hotel london ...",Neutral
1,81667,NaN,CNA,NaN,nice claim top spot in ligue 1 with late win a...,Nice moved into provisional first place in the...,https://www.channelnewsasia.com/sport/nice-cla...,https://onecms-res.cloudinary.com/image/upload...,2023-10-27 21:28:48.000000,Nice moved into provisional first place in the...,Monaco,nice moved into provisional first place in the...,Positive
2,81694,time,Time,Christina Larson / AP,amphibians are the world’s most vulnerable spe...,"The world’s frogs, salamanders, newts, and oth...",https://time.com/6320467/amphibians-most-vulne...,https://api.time.com/wp-content/uploads/2023/1...,2023-10-04 17:36:18.000000,"The worlds frogs, salamanders, newts and other...",Madagascar,"the world’s frogs, salamanders, newts and othe...",Negative
3,81695,NaN,Phys.Org,Sara Schmidt,image: rusty red waters in madagascar,Iron-rich sediment colors the red-orange water...,https://phys.org/news/2023-10-image-rusty-red-...,https://scx2.b-cdn.net/gfx/news/2023/image-rus...,2023-10-31 18:04:02.000000,Iron-rich sediment colors the red-orange water...,Madagascar,iron-rich sediment colors the red-orange water...,Neutral
4,81703,NaN,Digital Trends,Jason Struss,everything leaving max (formerly hbo max) in n...,From Gangs of London to Fear the Walking Dead ...,https://www.digitaltrends.com/movies/everythin...,https://www.digitaltrends.com/wp-content/uploa...,2023-10-23 23:09:18.000000,"Everything ends. No, I’m not having an existen...",Madagascar,"everything ends. no, i’m not having an existen...",Neutral


In [43]:
country = [
    "Afghanistan", "Albania", "Algeria", "Andorra", "Angola", "Antigua and Barbuda", "Argentina", "Armenia",
    "Australia", "Austria", "Azerbaijan", "Bahamas", "Bahrain", "Bangladesh", "Barbados", "Belarus", "Belgium",
    "Belize", "Benin", "Bhutan", "Bolivia", "Bosnia and Herzegovina", "Botswana", "Brazil", "Brunei", "Bulgaria",
    "Burkina Faso", "Burundi", "Cabo Verde", "Cambodia", "Cameroon", "Canada", "Central African Republic", "Chad",
    "Chile", "China", "Colombia", "Comoros", "Congo", "Costa Rica", "Cote d'Ivoire", "Croatia",
    "Cuba", "Cyprus", "Czech", "Democratic Republic of the Congo", "Denmark", "Djibouti",
    "Dominica", "Dominican Republic", "Ecuador", "Egypt", "El Salvador", "Equatorial Guinea", "Eritrea", "Estonia",
    "Eswatini", "Ethiopia", "Fiji", "Finland", "France", "Gabon", "Gambia", "Georgia", "Germany",
    "Ghana", "Greece", "Grenada", "Guatemala", "Guinea", "Guinea-Bissau", "Guyana", "Haiti", "Honduras", "Hong Kong", "Hungary",
    "Iceland", "India", "Indonesia", "Iran", "Iraq", "Ireland", "Israel", "Italy", "Jamaica", "Japan", "Jordan",
    "Kazakhstan", "Kenya", "Kiribati", "Kosovo", "Kuwait", "Kyrgyzstan", "Laos", "Latvia", "Lebanon", "Lesotho",
    "Liberia", "Libya", "Liechtenstein", "Lithuania", "Luxembourg", "Madagascar", "Malawi", "Malaysia", "Maldives",
    "Mali", "Malta", "Marshall Islands", "Martinique", "Mauritania", "Mauritius", "Mexico", "Micronesia", "Moldova", "Monaco",
    "Mongolia", "Montenegro", "Morocco", "Mozambique", "Myanmar", "Namibia", "Nauru", "Nepal",
    "Netherlands", "New Zealand", "Nicaragua", "Niger", "Nigeria", "North Korea", "North Macedonia",
    "Norway", "Oman", "Pakistan", "Palau", "Palestine State", "Panama", "Papua New Guinea", "Paraguay", "Peru",
    "Philippines", "Poland", "Portugal", "Qatar", "Romania", "Russia", "Rwanda", "Saint Kitts and Nevis",
    "Saint Lucia", "Saint Vincent and the Grenadines", "Samoa", "San Marino", "Sao Tome and Principe", "Saudi Arabia",
    "Senegal", "Serbia", "Seychelles", "Sierra Leone", "Singapore", "Slovakia", "Slovenia", "Solomon Islands",
    "Somalia", "South Africa", "South Korea", "South Sudan", "Spain", "Sri Lanka", "Sudan", "Suriname", "Sweden",
    "Switzerland", "Syria", "Taiwan", "Tajikistan", "Tanzania", "Thailand", "Timor-Leste", "Togo", "Tonga",
    "Trinidad and Tobago", "Tunisia", "Turkey", "Turkmenistan", "Tuvalu", "Uganda", "Ukraine", "United Arab Emirates",
    "United Kingdom","UK", "United States of America","USA","United States", "Uruguay", "Uzbekistan", "Vanuatu", "Vatican City",
    "Venezuela", "Vietnam", "Yemen", "Zambia", "Zimbabwe"
]

In [59]:
# Mapping the categories to broader categories
mapping = {
    'Breaking News': [
        'COVID', 'News', 'Breaking News', 'nan'  # General breaking news items
    ],
    'Politics': [
        'Politics', 'Palestine, State of', 'United States', 'Taiwan, Province of China', 
        'Uruguay', 'Virgin Islands, U.S.', 'Viet Nam', 'Korea, Republic of',
        'Russian Federation', 'Iran, Islamic Republic of'
    ],
    'World News': [
        'world', 'country', 'Congo, The Democratic Republic of the', 'America', 
        'Africa', 'Europe', 'Asia', 'Guam', 'Aruba', 'Guernsey', 'Antarctica',
        'Bermuda', "Côte d'Ivoire", 'Christmas Island', 'Cayman Islands', 
        'Gibraltar', 'Greenland', 'Isle of Man', 'Jersey', 'Macao', 'Réunion', 
        'Montserrat', 'Puerto Rico', 'Martinique'
    ],
    'Business/Finance': [
        'Business/Finance', 'Real estate', 'Stock', 'Startups', 'Finance', 'Entrepreneurship',
        'Cryptocurrency', 'Blockchain', 'Bitcoin', 'Amazon', 'Jobs'
    ],
    'Technology': [
        'Technology', 'Artificial Intelligence', 'Google', 'YouTube', 'Facebook', 
        'TikTok', 'Instagram', 'Virtual Reality', 'Coding', 'Design'
    ],
    'Science': [
        'Science', 'Space', 'Astronomy'
    ],
    'Food' : ['Food'],
    'Health': [
        'Health', 'Fitness', 'Nutrition', 'Vegan', 'Mindfulness', 'Meditation',
        'Yoga', 'Happiness', 'Philosophy', 'Psychology', 'Parenting', 'Motivation'
    ],
    'Entertainment': [
        'Entertainment', 'Movies', 'Music', 'Anime', 'Podcasts', 'Poetry', 
        'Games', 'Art', 'Photography', 'Beauty', 'Fashion', 'Love'
    ],
    'Sports': [
        'Sports', 'Hiking', 'Cars'
    ],
    'Environment': [
        'Environment', 'Climate', 'Gardening', 'Sustainability'
    ],
    'Crime': [
        # Since no explicit categories fit into 'Crime', we leave it empty for now.
    ],
    'Education': [
        'Education', 'History', 'Home', 'Recipes', 'DIY'
    ],
    'Weather': [
        'Weather'
    ],
    'Other': [
        'Other', 'Architecture', 'Minimalism', 'Productivity', 'Travel', 'Mindfulness', 
        'Christmas Island', 'Isle of Man', 'Guam', 'Aruba', 'Guernsey', 'Antarctica',
        'Bermuda', 'Côte d\'Ivoire', 'Christmas Island', 'Cayman Islands', 
        'Gibraltar', 'Greenland', 'Jersey', 'Macao', 'Réunion', 'Relationships', 'Pets'
    ]
}


In [60]:
def categ(x):
    x = str(x)
    cntry = [i.lower() for i in country]
    if x.lower() in cntry:
        x =  'country'

    for i in mapping.keys():
        if x in mapping[i]:
            return i
    return f'none yet - {x}'

In [5]:
import Preprocess

In [6]:
dir (Preprocess)

['Preprocess',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__']

## Classification of Headlines

In [7]:
from Preprocess.Preprocess import  create_tags

In [9]:
rating_df['tags'] = rating_df['category'].apply(create_tags)

In [11]:
rating_df.head(2)

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,article,title_sentiment,tags
0,81664,NaN,Forbes,"Elizabeth Brownfield, Contributor, \n Elizabet...",superstar chef yannick alléno brings refined f...,Now open in Mayfair at Four Seasons Hotel Lond...,https://www.forbes.com/sites/elizabethbrownfie...,https://imageio.forbes.com/specials-images/ima...,2023-11-01 03:27:21.000000,"Pavyllon London, at Four Seasons Hotel London ...",Monaco,"pavyllon london, at four seasons hotel london ...",Neutral,World News
1,81667,NaN,CNA,NaN,nice claim top spot in ligue 1 with late win a...,Nice moved into provisional first place in the...,https://www.channelnewsasia.com/sport/nice-cla...,https://onecms-res.cloudinary.com/image/upload...,2023-10-27 21:28:48.000000,Nice moved into provisional first place in the...,Monaco,nice moved into provisional first place in the...,Positive,World News


#### Approach 1 Regression with TFIDF

In [48]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder


In [29]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
# Define custom stop words list
custom_stop_words = set(stopwords.words('english'))
#  Function to preprocess text
def preprocess_text_lemmatize(text):
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    # Initialize WordNet Lemmatizer
    lemmatizer = WordNetLemmatizer()

    # Define custom stop words list
    custom_stop_words = set(stopwords.words('english'))
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words and non-alphabetic tokens
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in custom_stop_words]
    
    # Lemmatize tokens
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    
    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

In [30]:
# Function to preprocess text using stemming
def preprocess_text_stemming(text):
    import nltk
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    from nltk.stem import WordNetLemmatizer
    from nltk.stem import PorterStemmer

    # Tokenize text
    tokens = word_tokenize(text)
    
    # Remove stop words and non-alphabetic tokens
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in custom_stop_words]
    
    # Initialize Porter Stemmer
    stemmer = PorterStemmer()
    # Stem tokens
    tokens = [stemmer.stem(word) for word in tokens]
    
    # Join tokens back into text
    preprocessed_text = ' '.join(tokens)
    
    return preprocessed_text

#### for 1 
- get the text and tags to a new df
- encode tags
- split to train, test 
- method 1 
- - tdidf fit and save
- - fit on train
- - transform with tidf
- - predict on text
- - evaluate

- Method 2

In [27]:
df =   rating_df[['title','article','tags']]

In [32]:
# nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\otto\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt_tab.zip.


True

In [34]:
df.loc[:,'title_clean'] = df['title'].apply(preprocess_text_lemmatize)

In [35]:
df.loc[:,'article_clean'] = df['article'].apply(preprocess_text_stemming)

C:\Users\otto\AppData\Local\Temp\ipykernel_4032\1571461915.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'article_clean'] = df['article'].apply(preprocess_text_stemming)


In [39]:
df.head(1)

,title,article,tags,title_clean,article_clean,text
0,superstar chef yannick alléno brings refined f...,"pavyllon london, at four seasons hotel london ...",World News,superstar chef yannick alléno brings refined f...,pavyllon london four season hotel london park ...,superstar chef yannick alléno brings refined f...


In [38]:
df.loc[:,'text'] = df['title_clean'] + " " + df['article_clean']

C:\Users\otto\AppData\Local\Temp\ipykernel_4032\1113171239.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[:,'text'] = df['title_clean'] + " " + df['article_clean']


In [50]:
df.tags

0        World News
1        World News
2        World News
3        World News
4        World News
            ...    
58351     Education
58352     Education
58353     Education
58354     Education
58355     Education
Name: tags, Length: 58356, dtype: object

In [53]:
OHEencoder = OneHotEncoder()
y_encoded = OHEencoder.fit_transform(df[['tags']])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['text'],y_encoded, test_size=0.2, random_state=42)



In [42]:
# rating_df

In [54]:
# Initialize TF-IDF Vector
# Fit the TF-IDF vectorizer on the entire corpus
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
vectorizer.fit(df['text'])


TfidfVectorizer(max_features=5000, stop_words='english')

In [55]:
# Transform the training and test data separately
X_train_tfidf = vectorizer.transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)




#### Logistic Regression

In [45]:
# Proceed with model training as usual
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)

LogisticRegression(max_iter=1000)

In [46]:
# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)


In [47]:
# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))

Accuracy: 58.81%
              precision    recall  f1-score   support

           0       0.52      0.25      0.34       524
           1       0.66      0.63      0.64      1959
           2       0.53      0.24      0.33       482
           3       0.49      0.43      0.46       642
           4       0.43      0.26      0.33       241
           5       0.48      0.23      0.31       231
           6       0.59      0.39      0.47       597
           7       0.49      0.24      0.32       447
           8       0.49      0.23      0.32       231
           9       0.50      0.23      0.32       287
          10       0.63      0.30      0.40       294
          11       0.55      0.53      0.54       990
          12       0.57      0.30      0.39       202
          13       0.60      0.85      0.70      4545

    accuracy                           0.59     11672
   macro avg       0.54      0.36      0.42     11672
weighted avg       0.58      0.59      0.56     11672



In [19]:
print(rating_df['article'][1])

nice moved into provisional first place in the ligue 1 standings thanks to a second-half hicham boudaoui goal that secured a 1-0 win at lowly clermont foot on friday. the result lifted nice to first place with 22 points, two points ahead of as monaco, who play at lille on sunday. clermont are second-bottom with five points. boudaoui struck in the 74th minute after running unmarked into the box and tapping home a well-placed cross. nice had a chance to open the scoring just before halftime when alidu seidu fouled boudaoui in the area but gaetan labord's penalty went wide, leaving the first half scoreless. francesco farioli's nice have had an exceptional start to the season, maintaining an unbeaten record and securing seven clean sheets, the best in europe's top five leagues.  


In [20]:
print(rating_df['title'][1])

nice claim top spot in ligue 1 with late win at clermont


In [22]:
print(rating_df['description'][1])

Nice moved into provisional first place in the Ligue 1 standings thanks to a second-half Hicham Boudaoui goal that secured a 1-0 win at lowly Clermont Foot on Friday.The result lifted Nice to first place with 22 points, two points ahead of AS Monaco, who play…


In [14]:
# Check for any missing values and drop them
rating_df.dropna(subset=[''])#inplace=True)

,article_id,source_id,source_name,author,title,description,url,url_to_image,published_at,content,category,article,title_sentiment,tags
2,81694,time,Time,Christina Larson / AP,amphibians are the world’s most vulnerable spe...,"The world’s frogs, salamanders, newts, and oth...",https://time.com/6320467/amphibians-most-vulne...,https://api.time.com/wp-content/uploads/2023/1...,2023-10-04 17:36:18.000000,"The worlds frogs, salamanders, newts and other...",Madagascar,"the world’s frogs, salamanders, newts and othe...",Negative,World News
5,81707,al-jazeera-english,Al Jazeera English,Al Jazeera,unga calls for humanitarian truce in israel-ha...,The resolution passed with 120 votes in favour...,https://www.aljazeera.com/news/2023/10/27/unga...,https://www.aljazeera.com/wp-content/uploads/2...,2023-10-27 22:16:05.000000,The United Nations General Assembly has passed...,Madagascar,the resolution passed with 120 votes in favour...,Neutral,World News
6,81717,bbc-news,BBC News,https://www.facebook.com/bbcnews,malawi heatwave warning issued as temperatures...,"Temperatures could soar to 44C, the weather bu...",https://www.bbc.co.uk/news/world-africa-67086254,https://ichef.bbci.co.uk/news/1024/branded_new...,2023-10-12 10:40:48.000000,Malawians have been warned of an extreme heatw...,Madagascar,malawians have been warned of an extreme heatw...,Neutral,World News
11,81739,rt,RT,RT,russian marine scientists to conduct african r...,Russian scientists from VNIRO will conduct res...,https://www.rt.com/africa/585847-russian-scien...,https://mf.b37mrtl.ru/files/2023.10/article/65...,2023-10-26 11:04:42.000000,Scientists from the All-Russian Research Insti...,Madagascar,scientists from the all-russian research insti...,Neutral,World News
16,81755,al-jazeera-english,Al Jazeera English,Al Jazeera,‘kids whose insides are destroyed’: the return...,Mauritius grapples with a drug epidemic of unp...,https://www.aljazeera.com/features/2023/10/4/k...,https://www.aljazeera.com/wp-content/uploads/2...,2023-10-04 14:43:28.000000,"Port Louis, Mauritius Until early 2022, Gaetan...",Madagascar,the island nation is grappling with a drug epi...,Negative,World News
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
58312,779778,the-times-of-india,The Times of India,Akash Podishetti,"Alipay exits Zomato via 3.44% stake sale, take...",Bank of America and Morgan Stanley acted as ad...,https://economictimes.indiatimes.com/markets/s...,"https://img.etimg.com/thumb/msid-105599369,wid...",2023-11-29 14:08:38,China's Alipay sold its entire stake in food d...,America,Reuters China's Alipay sold its entire stake...,Neutral,World News
58338,780725,the-times-of-india,The Times of India,AFP,Google opens biggest European cyber centre,Google opened its largest cybersecurity centre...,https://economictimes.indiatimes.com/tech/tech...,"https://img.etimg.com/thumb/msid-105598533,wid...",2023-11-29 13:22:51,"Malaga, Nov 29, 2023 -Google opened its larges...",Europe,"Reuters Malaga, Nov 29, 2023 -Google opened it...",Positive,World News
58342,780872,wired,Wired,Joe Ray,"Best AeroPress Coffee Makers (2023): Original,...",The portable coffee makers from AeroPress come...,https://www.wired.com/story/best-aeropress/,https://media.wired.com/photos/656650a24cb6508...,2023-11-29 14:00:00,Buying an AeroPress used to be easy; for years...,Home,"Buying an AeroPressused to be easy; for years,...",Positive,Education
58345,781011,al-jazeera-english,Al Jazeera English,Ruwaida Amer,"‘Save what remains of Gaza’, hospital director...",The burden on the few remaining hospitals oper...,https://www.aljazeera.com/news/2023/11/29/save...,https://www.aljazeera.com/wp-content/uploads/2...,2023-11-29 13:58:04,"Khan Younis, Gaza Strip Ahmed Isleem wishes he...",Home,The burden on the few remaining hospitals oper...,Neutral,Education


In [38]:
rating_df.columns

Index(['article_id', 'source_id', 'source_name', 'author', 'title',
       'description', 'url', 'url_to_image', 'published_at', 'content',
       'category', 'article', 'title_sentiment'],
      dtype='object')

In [ ]:



# Encode the tags into numerical values
label_encoder = LabelEncoder()
df['tag'] = label_encoder.fit_transform(df['tag'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(df['headline'], df['tag'], test_size=0.2, random_state=42)

# Initialize TF-IDF Vectorizer
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')

# Fit and transform the training data
X_train_tfidf = vectorizer.fit_transform(X_train)

# Transform the test data
X_test_tfidf = vectorizer.transform(X_test)

# Initialize the Logistic Regression model
model = LogisticRegression(max_iter=1000)

# Train the model
model.fit(X_train_tfidf, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test_tfidf)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy * 100:.2f}%')

# Print the classification report
print(classification_report(y_test, y_pred))

